In [ ]:
# ⚙️ Automatic VisDrone dataset download + YOLO conversion

from ultralytics.utils.downloads import download
from ultralytics.utils import TQDM
from pathlib import Path
from PIL import Image
import shutil
import os

# Function: Convert VisDrone to YOLO format
def visdrone2yolo(dir, split, source_name=None):
    """Convert VisDrone annotations to YOLO format."""
    source_dir = dir / (source_name or f"VisDrone2019-DET-{split}")
    images_dir = dir / "images" / split
    labels_dir = dir / "labels" / split
    labels_dir.mkdir(parents=True, exist_ok=True)

    if (source_images_dir := source_dir / "images").exists():
        images_dir.mkdir(parents=True, exist_ok=True)
        for img in source_images_dir.glob("*.jpg"):
            img.rename(images_dir / img.name)

    for f in TQDM((source_dir / "annotations").glob("*.txt"), desc=f"Converting {split}"):
        img_size = Image.open(images_dir / f.with_suffix(".jpg").name).size
        dw, dh = 1.0 / img_size[0], 1.0 / img_size[1]
        lines = []

        with open(f, encoding="utf-8") as file:
            for row in [x.split(",") for x in file.read().strip().splitlines()]:
                if row[4] != "0":  # Skip ignored regions
                    x, y, w, h = map(int, row[:4])
                    cls = int(row[5]) - 1
                    x_center, y_center = (x + w / 2) * dw, (y + h / 2) * dh
                    w_norm, h_norm = w * dw, h * dh
                    lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

        (labels_dir / f.name).write_text("".join(lines), encoding="utf-8")


# Dataset directory
dir = Path("datasets/VisDrone")

# Download URLs from Ultralytics
urls = [
    "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-train.zip",
    "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-val.zip",
    "https://github.com/ultralytics/assets/releases/download/v0.0.0/VisDrone2019-DET-test-dev.zip",
]

# Download all three parts
print("📥 Downloading VisDrone dataset (~2GB total)...")
download(urls, dir=dir, threads=4)

# Convert dataset to YOLO format
splits = {"VisDrone2019-DET-train": "train", "VisDrone2019-DET-val": "val", "VisDrone2019-DET-test-dev": "test"}
for folder, split in splits.items():
    visdrone2yolo(dir, split, folder)
    shutil.rmtree(dir / folder)

print("✅ VisDrone dataset ready in YOLO format!")
